<a href="https://colab.research.google.com/github/IzzulGod/GPT2-Indo-Instruct-Tuned/blob/main/sharegpt_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets peft accelerate
!pip install -U datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("IzzulGod/GPT2-Indo-Instruct-Tuned")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    "IzzulGod/GPT2-Indo-Instruct-Tuned",
    torch_dtype=torch.float16,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset, Dataset

dataset = load_dataset("FreedomIntelligence/sharegpt-indonesian")

def convert_sharegpt_to_single_turn(examples):
    results = []
    for convo in examples["conversations"]:
        for i in range(len(convo) - 1):
            m1 = convo[i]
            m2 = convo[i + 1]

            if m1["from"] == "human" and m2["from"] == "gpt":
                user_msg = m1["value"].strip()
                bot_msg = m2["value"].strip()
                text = f"User: {user_msg}\nAI: {bot_msg} <|endoftext|>"
                results.append({"text": text})
    return {"text": results}

dataset = dataset.map(
    convert_sharegpt_to_single_turn,
    batched=True,
    remove_columns=["conversations", "id", "lang"]
)

for i in range(3):
    print(f"\n🔹 Sampel {i+1}")
    print("=" * 40)
    print(dataset["train"][i]["text"])


README.md:   0%|          | 0.00/208 [00:00<?, ?B/s]

sharegpt-indonesian.json:   0%|          | 0.00/31.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5560 [00:00<?, ? examples/s]

Map:   0%|          | 0/5560 [00:00<?, ? examples/s]


🔹 Sampel 1
{'text': 'User: Tunjukkan saya daftar pentingnya SEO bagi pemula.\nAI: Tentu, berikut adalah daftar dasar-dasar SEO untuk pemula:\n1. Penelitian kata kunci: Ini melibatkan mengidentifikasi kata kunci dan frasa yang dicari oleh audiens target Anda di mesin pencari.\n2. Optimasi halaman: Ini meliputi mengoptimalkan konten situs web Anda, gambar, tag judul, deskripsi meta, dan struktur URL untuk membuatnya lebih bersahabat dengan mesin pencari.\n3. Optimalisasi teknis: Ini melibatkan meningkatkan elemen teknis situs web Anda seperti kecepatan situs, kecocokan dengan perangkat seluler, dan memastikan mudah bagi mesin pencari untuk merayapi dan mengindeks.\n4. Bangunan tautan: Ini melibatkan memperoleh tautan balik berkualitas tinggi dari situs web lain untuk meningkatkan otoritas dan peringkat situs web Anda.\n5. Pembuatan konten: Membuat konten berkualitas tinggi dan relevan yang menjawab niat pencarian audiens target Anda penting untuk meningkatkan visibilitas situs web Anda 

In [4]:
sample = dataset["train"][:2]
print(sample)


{'text': [{'text': 'User: Tunjukkan saya daftar pentingnya SEO bagi pemula.\nAI: Tentu, berikut adalah daftar dasar-dasar SEO untuk pemula:\n1. Penelitian kata kunci: Ini melibatkan mengidentifikasi kata kunci dan frasa yang dicari oleh audiens target Anda di mesin pencari.\n2. Optimasi halaman: Ini meliputi mengoptimalkan konten situs web Anda, gambar, tag judul, deskripsi meta, dan struktur URL untuk membuatnya lebih bersahabat dengan mesin pencari.\n3. Optimalisasi teknis: Ini melibatkan meningkatkan elemen teknis situs web Anda seperti kecepatan situs, kecocokan dengan perangkat seluler, dan memastikan mudah bagi mesin pencari untuk merayapi dan mengindeks.\n4. Bangunan tautan: Ini melibatkan memperoleh tautan balik berkualitas tinggi dari situs web lain untuk meningkatkan otoritas dan peringkat situs web Anda.\n5. Pembuatan konten: Membuat konten berkualitas tinggi dan relevan yang menjawab niat pencarian audiens target Anda penting untuk meningkatkan visibilitas situs web Anda di

In [5]:
def tokenize(batch):
    texts = [item["text"] for item in batch["text"]]
    tokens = tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


tokenized_dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns=["text"]
)


sample_texts = [dataset["train"][i]["text"]["text"] for i in range(3)]
tokenized_batch = tokenizer(
    sample_texts,
    truncation=True,
    max_length=512,
    padding="max_length"
)

for i in range(3):
    print(f"\n🔹 Sampel {i+1}")
    print("=" * 40)
    print("📄 Teks Asli:\n")
    print(sample_texts[i])
    print("\n🔢 Token IDs:\n")
    print(tokenized_batch["input_ids"][i])
    print("\n🔤 Decoded dari Token:\n")
    print(tokenizer.decode(tokenized_batch["input_ids"][i]))
    print("=" * 40)


Map:   0%|          | 0/16580 [00:00<?, ? examples/s]


🔹 Sampel 1
📄 Teks Asli:

User: Tunjukkan saya daftar pentingnya SEO bagi pemula.
AI: Tentu, berikut adalah daftar dasar-dasar SEO untuk pemula:
1. Penelitian kata kunci: Ini melibatkan mengidentifikasi kata kunci dan frasa yang dicari oleh audiens target Anda di mesin pencari.
2. Optimasi halaman: Ini meliputi mengoptimalkan konten situs web Anda, gambar, tag judul, deskripsi meta, dan struktur URL untuk membuatnya lebih bersahabat dengan mesin pencari.
3. Optimalisasi teknis: Ini melibatkan meningkatkan elemen teknis situs web Anda seperti kecepatan situs, kecocokan dengan perangkat seluler, dan memastikan mudah bagi mesin pencari untuk merayapi dan mengindeks.
4. Bangunan tautan: Ini melibatkan memperoleh tautan balik berkualitas tinggi dari situs web lain untuk meningkatkan otoritas dan peringkat situs web Anda.
5. Pembuatan konten: Membuat konten berkualitas tinggi dan relevan yang menjawab niat pencarian audiens target Anda penting untuk meningkatkan visibilitas situs web Anda di

In [7]:
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig, TaskType

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["c_attn", "c_proj", "mlp.c_fc", "mlp.c_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 4,718,592 || all params: 129,158,400 || trainable%: 3.6533


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [8]:
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="./gpt2-chatbot-lora",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    logging_steps=100,
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=3e-4,
    warmup_ratio=0.03,
    fp16=True,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    report_to="none"
)

train_dataset = tokenized_dataset["train"]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
100,3.197400
200,3.110900
300,3.096000
400,3.086400
500,3.091800
600,3.025800
700,3.006300
800,2.960900
900,2.990600
1000,2.982000


TrainOutput(global_step=1038, training_loss=3.0520489496073053, metrics={'train_runtime': 2321.1509, 'train_samples_per_second': 14.286, 'train_steps_per_second': 0.447, 'total_flos': 9145115046051840.0, 'train_loss': 3.0520489496073053, 'epoch': 2.0})

In [8]:
model.save_pretrained("/content/drive/MyDrive/gpt2-lora-adapter")
tokenizer.save_pretrained("/content/drive/MyDrive/gpt2-lora-adapter")


('/content/drive/MyDrive/gpt2-lora-adapter/tokenizer_config.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/special_tokens_map.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/vocab.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/merges.txt',
 '/content/drive/MyDrive/gpt2-lora-adapter/added_tokens.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/tokenizer.json')

In [9]:
from peft import PeftModel

if isinstance(model, PeftModel):

    merged_model = model.merge_and_unload()

    merged_model.save_pretrained("/content/drive/MyDrive/gpt2-lora-merged")
    tokenizer.save_pretrained("/content/drive/MyDrive/gpt2-lora-merged")
else:
    print("tidak ada adapter untuk digabungkan")


In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/content/drive/MyDrive/gpt2-lora-merged"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda")

prompt = "User: Siapa presiden pertama Indonesia?\nAI:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


User: Siapa presiden pertama Indonesia?
AI: Presiden pertama Indonesia adalah Sukarno. Sukarno adalah seorang presiden yang dikenal karena jasanya dalam mengembangkan dan membangun sistem pertahanan kemerdekaan di Indonesia. Dia terkenal karena kontribusinya dalam pengembangan teknologi militer, pengembangan alat-alat baru seperti rudal, peluru kendali, dan bom atom. Dia juga memimpin gerakan perlawanan terhadap invasi Jepang pada tahun 1942, terutama selama Perang Pasifik. 
Salah satu peran terbesarnya adalah membuat keputusan tentang konflik antara Belanda dan Amerika Serikat. Sukarno adalah pemimpin utama dari gerakan perlawanan terhadap pendudukan Jepang dan berhasil melobi untuk mendukung perjuangan yang dilakukan oleh Amerika Serikat dalam Perang Dunia II. 
Sebagai presiden pertama Indonesia, Sukarno adalah salah satu pemimpin kunci dalam gerakan perlawanan terhadap invasi Jepang di
